# GAN 모델 이용해서 MNIST 손글씨 무작위 생성

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data\t10k-images-idx3-ubyte.gz
Extracting ./mnist/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# 하이퍼 파라미터 생성
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28  # 실제 이미지 크기
n_noise = 128  # 생성자의 입력값으로 사용할 노이즈의 크기
# 즉, 랜덤한 노이즈를 입력하고 그 노이즈에서 손글씨 이미지를 무작위로 생성해내도록 한다.

In [5]:
# GAN도 비지도 학습이므로 오토인코더처럼 Y를 사용하지 않는다.
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise]) # 가짜 이미지용 플레이스홀더

In [6]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01)) #가중치
G_b2 = tf.Variable(tf.zeros([n_input])) #편향(바이어스)

In [7]:
# 구분자 신경망에 사용할 변수들 설정
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))  # 0~1사이 값 출력으로, 스칼라 값 출력하도록 구성
D_b2 = tf.Variable(tf.zeros([1]))

In [8]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    
    return output

# 생성자는 무작위로 생성한 노이즈를 받아 가중치와 편향 반영해 은닉층 생성 -> 은닉층에서 실제 이미지와 같은 크기의 결괏값을 출력하는 구성

In [10]:
# 구분자 신경망도 같은 구성이지만 0~1 사이의 스칼라값을 출력하도록 생성
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    
    return output

In [11]:
# 무작위한 노이즈 만들어주는 함수
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [16]:
# 노이즈Z를 이용해 가짜 이미지 생성자 G 만들고, G가 만든 가짜이미지와, X의 진짜 이미지를 각각 구분자에 넣어 진짜인지 확인
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [17]:
# 손실값 구하기. 가짜 손실값, 진짜 손실값
# 경찰 학습시 진짜이미지 판별값은 D_real은 1에 가까워짐(진짜라고 판별)
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

In [18]:
# D_real과 1에서 D_gene를 뺀 값을 더한 값을 손실값으로 하여, 이 값을 최대화하면 경찰 학습이 이뤄진다.
# 위조지폐범 학습은 가짜 이미지 판별값 D_gene를 1에 가깝게 만들면 된다. 즉, 가짜 이미지를 넣어도 진짜같다고 판별해야 한다.
loss_G = tf.reduce_mean(tf.log(D_gene))

In [19]:
# 손실값들을 이용해 학습. 각 생성자 신경망에 사용되는 변수들만 사용해 최적화해야 한다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]
# 서로 경쟁관계에 있기 때문이다.

In [21]:
#GAN 논문에 따라 loss를 최대화 해 변수들을 최적화 한다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [22]:
# 학습시키는 코드
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

# 노이즈를 생성해주는 get_noise 함수를 통해 배치 크기만큼 노이즈를 만들고 이를 입력해준다.
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
        
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
# GAN 모델 완성 !

Epoch: 0000 D loss: -0.4272 G loss: -2.131
Epoch: 0001 D loss: -0.3348 G loss: -2.264
Epoch: 0002 D loss: -0.2304 G loss: -2.501
Epoch: 0003 D loss: -0.3105 G loss: -1.813
Epoch: 0004 D loss: -0.217 G loss: -2.481
Epoch: 0005 D loss: -0.1334 G loss: -3.34
Epoch: 0006 D loss: -0.16 G loss: -3.367
Epoch: 0007 D loss: -0.2907 G loss: -2.457
Epoch: 0008 D loss: -0.07033 G loss: -3.489
Epoch: 0009 D loss: -0.1968 G loss: -3.082
Epoch: 0010 D loss: -0.2034 G loss: -3.193
Epoch: 0011 D loss: -0.1853 G loss: -3.03
Epoch: 0012 D loss: -0.3591 G loss: -2.692
Epoch: 0013 D loss: -0.2986 G loss: -2.551
Epoch: 0014 D loss: -0.4465 G loss: -2.429
Epoch: 0015 D loss: -0.3847 G loss: -2.521
Epoch: 0016 D loss: -0.3419 G loss: -2.548
Epoch: 0017 D loss: -0.3668 G loss: -2.517
Epoch: 0018 D loss: -0.3826 G loss: -2.283
Epoch: 0019 D loss: -0.4731 G loss: -2.433
Epoch: 0020 D loss: -0.4804 G loss: -2.722
Epoch: 0021 D loss: -0.5006 G loss: -2.371
Epoch: 0022 D loss: -0.4723 G loss: -2.324
Epoch: 0023 D l

In [27]:
####################################################
# 학습 결과를 확인하는 코드 즉, 확인용 이미지 생성 #
####################################################
if epoch == 0 or (epoch + 1) % 10 == 0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise)
    samples = sess.run(G, feed_dict={Z: noise})
    
    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
    
    for i in range(sample_size):
        ax[i].set_axis_off()
        ax[i].imshow(np.reshape(samples[i], (28, 28)))
        
    plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    plt.close(fig)
    
print('최적화 완료!')

최적화 완료!
